# BigDataProgramming TermProject

## 엔카 크롤링

### Driver Load

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [2]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

In [3]:
# 페이지 로딩 후 대기 시간 설정
wait_time = 1

### run
#### 매물 조회 리스트를 순회하며 무작위 `매물id` 수집
##### 중복되는 `매물id` 는 제외

In [4]:
# 페이지 순회를 위한 범위 지정 (예: 1~5페이지)
page_start = 1
page_end = 15  # 원하는 페이지 수까지 변경 가능

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page={page}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 최종 결과 출력
print("모든 페이지에서 추출된 번호들:")
for number in all_numbers:
    print(number)


페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
페이지 4 크롤링 완료, 현재까지 추출된 번호 수: 40
페이지 5 크롤링 완료, 현재까지 추출된 번호 수: 50
페이지 6 크롤링 완료, 현재까지 추출된 번호 수: 60
페이지 7 크롤링 완료, 현재까지 추출된 번호 수: 70
페이지 8 크롤링 완료, 현재까지 추출된 번호 수: 80
페이지 9 크롤링 완료, 현재까지 추출된 번호 수: 90
페이지 10 크롤링 완료, 현재까지 추출된 번호 수: 100
페이지 11 크롤링 완료, 현재까지 추출된 번호 수: 110
페이지 12 크롤링 완료, 현재까지 추출된 번호 수: 120
페이지 13 크롤링 완료, 현재까지 추출된 번호 수: 130
페이지 14 크롤링 완료, 현재까지 추출된 번호 수: 140
페이지 15 크롤링 완료, 현재까지 추출된 번호 수: 150
모든 페이지에서 추출된 번호들:
38420143
37309395
38019223
37541021
38565073
38580886
38525625
38387754
38418044
38475220
38358746
38054196
38333406
38347763
38473114
38148760
38062962
38418716
38554831
38054331
38629566
38291054
38139463
38299674
38556546
38375934
38536766
38510611
38309460
37816064
37777439
38261159
38285903
38146682
38554360
38585582
37592388
36914541
38608181
38311030
38162996
38132679
38262701
38518345
38157015
38585093
38576573
38487770
38559360
38321781
38043228
38338599
38396357
38619418
3854

### run
#### 수집한 `매물id` 를 `https://fem.encar.com/cars/report/accident/{매물id}` 에 대입해 보험이력 조회

In [5]:
try:
    for car_id in all_numbers:
        # 4. URL 접속
        url = f"https://fem.encar.com/cars/report/accident/{car_id}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요시 WebDriverWait으로 대체 가능
        
        # 5. 데이터 추출
        try:
            print("매물id:", car_id)
            elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/ul/li[1]/span")
            car_detail = ",".join([element.text for element in elements])
            print("일반사양:", car_detail)
            print("용도변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text)
            print("번호소유자변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text)
            print("특수사고이력:", driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text)
            print("보험사고_내차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[5]/span").text)
            print("보험사고_타차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[6]/span").text)

        finally:
            print()
        #     # 기본 정보
        #     일반사양 = driver.find_element(By.CLASS_NAME, "ReportAccidentSummary_txt__fVCew").text
        #     용도변경이력 = driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text
        #     번호소유자변경이력 = driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text
        #     특수사고이력 = driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text
        #     보험사고_내차 = driver.find_element(By.XPATH, "//li[contains(.,'보험사고 이력 (내차 피해)')]//span").text
        #     보험사고_타차 = driver.find_element(By.XPATH, "//li[contains(.,'보험사고 이력 (타차 가해)')]//span").text
            
        #     # 차량 스펙
        #     제작사 = driver.find_element(By.XPATH, "//th[contains(.,'제작사')]/following-sibling::td").text
        #     자동차명 = driver.find_element(By.XPATH, "//th[contains(.,'자동차명')]/following-sibling::td").text
        #     연식 = driver.find_element(By.XPATH, "//th[contains(.,'연식')]/following-sibling::td").text
        #     사용연료 = driver.find_element(By.XPATH, "//th[contains(.,'사용연료')]/following-sibling::td").text
        #     배기량 = driver.find_element(By.XPATH, "//th[contains(.,'배기량')]/following-sibling::td").text
        #     차체형상 = driver.find_element(By.XPATH, "//th[contains(.,'차체형상')]/following-sibling::td").text
        #     용도차종 = driver.find_element(By.XPATH, "//th[contains(.,'용도 및 차종')]/following-sibling::td").text
        #     최초보험가입일 = driver.find_element(By.XPATH, "//th[contains(.,'최초 보험 가입일자')]/following-sibling::td").text

        #     # 사용 이력
        #     영업용대여 = driver.find_element(By.XPATH, "//th[contains(.,'영업용(대여) 사용이력')]/following-sibling::td").text
        #     영업용일반 = driver.find_element(By.XPATH, "//th[contains(.,'영업용(일반) 사용이력')]/following-sibling::td").text
        #     관용용도 = driver.find_element(By.XPATH, "//th[contains(.,'관용용도 사용이력')]/following-sibling::td").text

        #     # 보험사고
        #     전손보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'전손 보험사고')]/following-sibling::td").text
        #     침수보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'침수 보험사고')]/following-sibling::td").text
        #     도난보험사고 = driver.find_element(By.XPATH, "//th[contains(.,'도난 보험사고')]/following-sibling::td").text

        #     # 6. 데이터 저장
        #     data.append({
        #         "매물등록번호": car_id,
        #         "일반사양": 일반사양,
        #         "용도변경이력": 용도변경이력,
        #         "번호/소유자변경이력": 번호소유자변경이력,
        #         "특수사고이력": 특수사고이력,
        #         "보험사고이력(내차피해)": 보험사고_내차,
        #         "보험사고이력(타차가해)": 보험사고_타차,
        #         "제작사": 제작사,
        #         "자동차명": 자동차명,
        #         "연식": 연식,
        #         "사용연료": 사용연료,
        #         "배기량": 배기량,
        #         "차체형상": 차체형상,
        #         "용도 및 차종": 용도차종,
        #         "최초 보험 가입일자": 최초보험가입일,
        #         "영업용(대여)사용이력": 영업용대여,
        #         "영업용(일반)사용이력": 영업용일반,
        #         "관용용도사용이력": 관용용도,
        #         "전손보험사고": 전손보험사고,
        #         "침수보험사고": 침수보험사고,
        #         "도난보험사고": 도난보험사고
        #     })
        # except Exception as e:
        #     print(f"ID {car_id}에서 데이터를 추출하는 중 오류 발생: {e}")

finally:
    # 7. 브라우저 닫기
    # driver.quit()
    print("finished")

매물id: 38420143
일반사양: BMW5시리즈 GT (F07)2014년식
용도변경이력: 없음
번호소유자변경이력: 0회2회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 2회3,226,771원
보험사고_타차: 없음

매물id: 37309395
일반사양: 닷지램픽업2021년식
용도변경이력: 없음
번호소유자변경이력: 0회1회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음

매물id: 38019223
일반사양: 기아올 뉴 K7 하이브리드2017년식세단 4도어
용도변경이력: 없음
번호소유자변경이력: 0회6회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음

매물id: 37541021
일반사양: 제네시스G802019년식세단 4도어
용도변경이력: 없음
번호소유자변경이력: 0회1회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 3회3,296,003원
보험사고_타차: 없음

매물id: 38565073
일반사양: 벤츠E-클래스 W2122016년식
용도변경이력: 없음
번호소유자변경이력: 0회3회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 3회6,642,530원
보험사고_타차: 2회/964,134원

매물id: 38580886
일반사양: 

finished


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[contains(.,'용도 변경이력')]//span"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100f27ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000100f20314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001009884b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001009cc898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100a060fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001009c10b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x00000001009c1d00 cxxbridge1$string$len + 324820
7   chromedriver                    

#### test

In [6]:
try:
    # 2. 테스트용 URL 설정
    maemul_id = "38554502"  # 매물 ID
    url = f"https://fem.encar.com/cars/report/accident/{maemul_id}"
    driver.get(url)

    # 3. 첫 번째 섹션 데이터 추출
    xpath1 = "/html/body/div[1]/div/div[2]/div[2]/div[1]"
    section1 = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath1)))
    section1_text = section1.text

    # 4. 두 번째 섹션 데이터 추출
    xpath2 = "/html/body/div[1]/div/div[2]/div[2]/div[2]"
    section2 = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath2)))
    section2_text = section2.text

    # 5. 결과 출력
    # print("첫 번째 섹션 데이터:")
    # print(section1_text)
    # print("\n두 번째 섹션 데이터:")
    # print(section2_text)

    # 기본 정보
    elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/ul/li[1]/span")
    car_detail = ",".join([element.text for element in elements])
    print("일반사양:", car_detail)
    print("용도변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'용도 변경이력')]//span").text)
    print("번호소유자변경이력:", driver.find_element(By.XPATH, "//li[contains(.,'번호 / 소유자 변경이력')]//span").text)
    print("특수사고이력:", driver.find_element(By.XPATH, "//li[contains(.,'특수 사고이력')]//span").text)
    print("보험사고_내차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[5]/span").text)
    print("보험사고_타차:", driver.find_element(By.XPATH, "//div/div[2]/div[2]/div[1]/ul/li[6]/span").text)
    
    # 차량 스펙
    print("제작사:", driver.find_element(By.XPATH, "//th[contains(.,'제작사')]/following-sibling::td").text)
    print("자동차명:", driver.find_element(By.XPATH, "//th[contains(.,'자동차명')]/following-sibling::td").text)
    print("연식:", driver.find_element(By.XPATH, "//th[contains(.,'연식')]/following-sibling::td").text)
    print("사용연료:", driver.find_element(By.XPATH, "//th[contains(.,'사용연료')]/following-sibling::td").text)
    print("배기량:", driver.find_element(By.XPATH, "//th[contains(.,'배기량')]/following-sibling::td").text)
    print("차체형상:", driver.find_element(By.XPATH, "//th[contains(.,'차체형상')]/following-sibling::td").text)
    # print("용도차종:", driver.find_element(By.XPATH, "//th[contains(.,'용도 및 차종')]/following-sibling::td").text)
    # print("최초보험가입일:", driver.find_element(By.XPATH, "//th[contains(.,'최초 보험 가입일자')]/following-sibling::td").text)
    
    # 사용 이력
    print("영업용대여:", driver.find_element(By.XPATH, "//th[contains(.,'영업용(대여) 사용이력')]/following-sibling::td").text)
    print("영업용일반:", driver.find_element(By.XPATH, "//th[contains(.,'영업용(일반) 사용이력')]/following-sibling::td").text)
    print("관용용도:", driver.find_element(By.XPATH, "//th[contains(.,'관용용도 사용이력')]/following-sibling::td").text)
    
    # 보험사고
    print("전손보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'전손 보험사고')]/following-sibling::td").text)
    print("침수보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'침수 보험사고')]/following-sibling::td").text)
    print("도난보험사고:", driver.find_element(By.XPATH, "//th[contains(.,'도난 보험사고')]/following-sibling::td").text)

finally:
    # 6. 브라우저 닫기
    print()

일반사양: 현대아반떼 (CN7)2022년식세단 4도어
용도변경이력: 없음
번호소유자변경이력: 0회0회
특수사고이력: 전손 0회침수(전손,분손) 0회도난 0회
보험사고_내차: 없음
보험사고_타차: 없음
제작사: 
자동차명: 
연식: 
사용연료: 
배기량: 
차체형상: 
영업용대여: 
영업용일반: 
관용용도: 
전손보험사고: 
침수보험사고: 
도난보험사고: 



#### main